<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/data"

Mall_Customers.csv  pdf_to_excel.pdf


In [0]:
pip install tika

In [90]:
import tika
import re

tika.initVM()
from tika import parser

parsed = parser.from_file('/content/gdrive/My Drive/data/pdf_to_excel.pdf')
# print(parsed["metadata"])
content = parsed["content"]
content = content.replace("Microsoft Word - Documents.docx", "").strip()
content = re.sub("\n*File [0-9]+:", "|||||", content)
print(content)

||||| 
File Name: Instructions.docx 
File Size: 2 MB 
Download Link: https://www.sendspace.com/file/r59by6 
Delete File 
Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c
1f 
 ||||| 
File Name: excel instructions.xlsx 
File Size: 995.5 KB 
Download Link: https://www.sendspace.com/file/lv6xjf 
Delete File 
Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690
6 
 ||||| 
File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-
OCR.pdf 
File Size: 173.3 MB 
Download Link: https://www.sendspace.com/file/mfj1z7 
Delete File 
Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27
81


In [93]:
section = content.split("|||||")
section = list(filter(None, section))

d = {}

i = 1
for line in section:
  d["File "+str(i)] = line
  i = i+1

for k, v in d.items():
  print(v)
#   for line in v.splitlines():
#     print(line)    
#     print("----")
#     print(line.split(" : "))

 
File Name: Instructions.docx 
File Size: 2 MB 
Download Link: https://www.sendspace.com/file/r59by6 
Delete File 
Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c
1f 
 
 
File Name: excel instructions.xlsx 
File Size: 995.5 KB 
Download Link: https://www.sendspace.com/file/lv6xjf 
Delete File 
Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690
6 
 
 
File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-
OCR.pdf 
File Size: 173.3 MB 
Download Link: https://www.sendspace.com/file/mfj1z7 
Delete File 
Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27
81


[' ']
['File Name: Instructions.docx ']
['File Size: 2 MB ']
['Download Link: https://www.sendspace.com/file/r59by6 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c']
['1f ']
[' ']
[' ']
['File Name: excel instructions.xlsx ']
['File Size: 995.5 KB ']
['Download Link: https://www.sendspace.com/file/lv6xjf ']
['Delete File ']
['Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690']
['6 ']
[' ']
[' ']
['File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-']
['OCR.pdf ']
['File Size: 173.3 MB ']
['Download Link: https://www.sendspace.com/file/mfj1z7 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27']
['81']


In [0]:
file = open("/content/gdrive/My Drive/data/pdf_to_excel.txt","w") 
file.write(content)
file.close()

In [0]:
match = re.match(r"(^File [0-9]+):([^File [0-9]+].*)", content, re.I)
if match:
    print("match")
    items = match.groups()

items
# re.split("File d+", content)
# key, value = content.split("File ")
# my_dict = {key: value}

# dict(zip(*content.split(':')))
# dict(zip(*iter(content.split(':'))))

('File 1',)